# CH03 word2vec

- 현업에서의 다루는 말뭉치의 어휘 수는 굉장히 크다. 100만개의 어휘 > 100만 x 100만이라는 거대한 행렬을 만들게 된다.
- SVD nxn행렬에 적용하는 비용은 O(n^3)로 계산시간이 n의 3제곱에 비례한다는것. 걸리는 시간이 굉장히 오래 걸린다.

# 3.1.3. 신경망에서의 단어처리 
- 단어를 있는 그대로 처리하지않고 '고정길이의 벡터'로 변환한다. > one-hot 벡터로 변환
- 편향을 이용하지 않는 완전연결계층은 [행렬곱]에 해당한다. 이는 MATMUL 계층과 같아진다.

In [8]:
import numpy as np 
c = np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
h = np.matmul(c,W)
print(h)

[[0.11693372 2.04967322 0.40079162]]


In [10]:
import sys 
sys.path.append('..')
from common.layers import(MatMul)

c= np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
layer=MatMul(W)
h=layer.forward(c)
print(h)

[[-0.63645023 -0.06135501  1.28066164]]


# 3.2 단순한 word2vec
- 이번에 사용할 신경망은 word2vec에서 제안하는 CBOW모델이다. 
### CBOW
- CBOW 모델은 타켓을 추측하는 용도의 신경망이다. ( 타겟은 중앙단어이고, 그주변단어들이 맥락)
- CBOW의 입력층의 갯수는 [맥락]으로 고려할 단어의 갯수와 같은 수로 설정을한다. 
- 은닉층은 입력층의 완전연결 계층에 의해 변환된 값이 되는데, 입력층이 여러개 > 전체를 평균하여 은닉층의 뉴런을 설정하면된다.

In [11]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul


# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)

[[-2.1041123  -1.45116849  2.17097331 -1.10325612  0.82026571  1.74657842
   0.2626459 ]]


- 필요한 가중치 W_in, W_out 초기화
- 입력층을 처리하는 Matmul계층을 맥락 수만큼(2개) 생성.
- 출력층 쪽 matmul 계층은 1개만 생성한다. 

In [16]:
import sys
sys.path.append('..')
from common.util import preprocess

text = 'you say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


create contexts target 함수는 인수를 2개 받는다. 
1. 단어의 ID 배열
2. 맥락의 윈도우 크기 그리고 맥락과 타켓을 numpy 다차원 배열로 돌려준다.

In [17]:
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

In [19]:
contexts, target = create_contexts_target(corpus, window_size =1 )
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


In [20]:
# 맥락과 타깃의 각 원소는 ID. 원핫 표현으로 바꾸자.

In [21]:
def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

# 3.4 CBOW모델 구현

In [24]:
from common.layers import MatMul, SoftmaxWithLoss
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

        
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

## 3.4.1 학습코드 구현

일반적인 모델의 학습과 같다. 
- 학습데이터를 준비해 신경망에 입력한 다음 기울기를 구하고 가중치를 매개변수로 갱신한다. 

In [26]:
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import preprocess, create_contexts_target, convert_one_hot


window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])

| 에폭 1 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 2 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 3 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 4 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 5 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 6 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 7 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 8 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 9 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 10 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 11 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 12 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 13 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 14 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 15 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 16 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 17 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 18 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 19 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 20 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 21 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 22 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 23 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 24 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 25 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 26 |

| 에폭 222 |  반복 1 / 2 | 시간 1[s] | 손실 1.15
| 에폭 223 |  반복 1 / 2 | 시간 1[s] | 손실 1.15
| 에폭 224 |  반복 1 / 2 | 시간 1[s] | 손실 1.16
| 에폭 225 |  반복 1 / 2 | 시간 1[s] | 손실 1.14
| 에폭 226 |  반복 1 / 2 | 시간 1[s] | 손실 1.20
| 에폭 227 |  반복 1 / 2 | 시간 1[s] | 손실 1.08
| 에폭 228 |  반복 1 / 2 | 시간 1[s] | 손실 1.16
| 에폭 229 |  반복 1 / 2 | 시간 1[s] | 손실 1.12
| 에폭 230 |  반복 1 / 2 | 시간 1[s] | 손실 1.12
| 에폭 231 |  반복 1 / 2 | 시간 1[s] | 손실 1.08
| 에폭 232 |  반복 1 / 2 | 시간 1[s] | 손실 1.25
| 에폭 233 |  반복 1 / 2 | 시간 1[s] | 손실 1.00
| 에폭 234 |  반복 1 / 2 | 시간 1[s] | 손실 1.11
| 에폭 235 |  반복 1 / 2 | 시간 1[s] | 손실 1.16
| 에폭 236 |  반복 1 / 2 | 시간 1[s] | 손실 1.18
| 에폭 237 |  반복 1 / 2 | 시간 1[s] | 손실 0.99
| 에폭 238 |  반복 1 / 2 | 시간 1[s] | 손실 1.08
| 에폭 239 |  반복 1 / 2 | 시간 1[s] | 손실 1.10
| 에폭 240 |  반복 1 / 2 | 시간 1[s] | 손실 1.16
| 에폭 241 |  반복 1 / 2 | 시간 1[s] | 손실 1.16
| 에폭 242 |  반복 1 / 2 | 시간 1[s] | 손실 1.02
| 에폭 243 |  반복 1 / 2 | 시간 1[s] | 손실 1.06
| 에폭 244 |  반복 1 / 2 | 시간 1[s] | 손실 1.08
| 에폭 245 |  반복 1 / 2 | 시간 1[s] | 손실 1.15
| 에폭 246 |  반복 1

| 에폭 431 |  반복 1 / 2 | 시간 2[s] | 손실 0.57
| 에폭 432 |  반복 1 / 2 | 시간 2[s] | 손실 0.52
| 에폭 433 |  반복 1 / 2 | 시간 2[s] | 손실 0.64
| 에폭 434 |  반복 1 / 2 | 시간 2[s] | 손실 0.62
| 에폭 435 |  반복 1 / 2 | 시간 2[s] | 손실 0.64
| 에폭 436 |  반복 1 / 2 | 시간 2[s] | 손실 0.71
| 에폭 437 |  반복 1 / 2 | 시간 2[s] | 손실 0.65
| 에폭 438 |  반복 1 / 2 | 시간 2[s] | 손실 0.54
| 에폭 439 |  반복 1 / 2 | 시간 2[s] | 손실 0.78
| 에폭 440 |  반복 1 / 2 | 시간 2[s] | 손실 0.45
| 에폭 441 |  반복 1 / 2 | 시간 2[s] | 손실 0.64
| 에폭 442 |  반복 1 / 2 | 시간 2[s] | 손실 0.68
| 에폭 443 |  반복 1 / 2 | 시간 2[s] | 손실 0.62
| 에폭 444 |  반복 1 / 2 | 시간 2[s] | 손실 0.62
| 에폭 445 |  반복 1 / 2 | 시간 2[s] | 손실 0.56
| 에폭 446 |  반복 1 / 2 | 시간 2[s] | 손실 0.61
| 에폭 447 |  반복 1 / 2 | 시간 2[s] | 손실 0.76
| 에폭 448 |  반복 1 / 2 | 시간 2[s] | 손실 0.38
| 에폭 449 |  반복 1 / 2 | 시간 2[s] | 손실 0.79
| 에폭 450 |  반복 1 / 2 | 시간 2[s] | 손실 0.61
| 에폭 451 |  반복 1 / 2 | 시간 2[s] | 손실 0.58
| 에폭 452 |  반복 1 / 2 | 시간 2[s] | 손실 0.46
| 에폭 453 |  반복 1 / 2 | 시간 2[s] | 손실 0.76
| 에폭 454 |  반복 1 / 2 | 시간 2[s] | 손실 0.54
| 에폭 455 |  반복 1

| 에폭 705 |  반복 1 / 2 | 시간 2[s] | 손실 0.39
| 에폭 706 |  반복 1 / 2 | 시간 2[s] | 손실 0.31
| 에폭 707 |  반복 1 / 2 | 시간 2[s] | 손실 0.46
| 에폭 708 |  반복 1 / 2 | 시간 2[s] | 손실 0.33
| 에폭 709 |  반복 1 / 2 | 시간 2[s] | 손실 0.49
| 에폭 710 |  반복 1 / 2 | 시간 2[s] | 손실 0.38
| 에폭 711 |  반복 1 / 2 | 시간 2[s] | 손실 0.29
| 에폭 712 |  반복 1 / 2 | 시간 2[s] | 손실 0.46
| 에폭 713 |  반복 1 / 2 | 시간 2[s] | 손실 0.40
| 에폭 714 |  반복 1 / 2 | 시간 2[s] | 손실 0.38
| 에폭 715 |  반복 1 / 2 | 시간 2[s] | 손실 0.50
| 에폭 716 |  반복 1 / 2 | 시간 2[s] | 손실 0.38
| 에폭 717 |  반복 1 / 2 | 시간 2[s] | 손실 0.30
| 에폭 718 |  반복 1 / 2 | 시간 2[s] | 손실 0.28
| 에폭 719 |  반복 1 / 2 | 시간 2[s] | 손실 0.47
| 에폭 720 |  반복 1 / 2 | 시간 2[s] | 손실 0.51
| 에폭 721 |  반복 1 / 2 | 시간 2[s] | 손실 0.25
| 에폭 722 |  반복 1 / 2 | 시간 2[s] | 손실 0.32
| 에폭 723 |  반복 1 / 2 | 시간 2[s] | 손실 0.45
| 에폭 724 |  반복 1 / 2 | 시간 2[s] | 손실 0.30
| 에폭 725 |  반복 1 / 2 | 시간 2[s] | 손실 0.46
| 에폭 726 |  반복 1 / 2 | 시간 2[s] | 손실 0.41
| 에폭 727 |  반복 1 / 2 | 시간 2[s] | 손실 0.37
| 에폭 728 |  반복 1 / 2 | 시간 2[s] | 손실 0.49
| 에폭 729 |  반복 1

| 에폭 931 |  반복 1 / 2 | 시간 3[s] | 손실 0.33
| 에폭 932 |  반복 1 / 2 | 시간 3[s] | 손실 0.20
| 에폭 933 |  반복 1 / 2 | 시간 3[s] | 손실 0.43
| 에폭 934 |  반복 1 / 2 | 시간 3[s] | 손실 0.30
| 에폭 935 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 936 |  반복 1 / 2 | 시간 3[s] | 손실 0.22
| 에폭 937 |  반복 1 / 2 | 시간 3[s] | 손실 0.53
| 에폭 938 |  반복 1 / 2 | 시간 3[s] | 손실 0.29
| 에폭 939 |  반복 1 / 2 | 시간 3[s] | 손실 0.11
| 에폭 940 |  반복 1 / 2 | 시간 3[s] | 손실 0.42
| 에폭 941 |  반복 1 / 2 | 시간 3[s] | 손실 0.32
| 에폭 942 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 943 |  반복 1 / 2 | 시간 3[s] | 손실 0.30
| 에폭 944 |  반복 1 / 2 | 시간 3[s] | 손실 0.30
| 에폭 945 |  반복 1 / 2 | 시간 3[s] | 손실 0.21
| 에폭 946 |  반복 1 / 2 | 시간 3[s] | 손실 0.42
| 에폭 947 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 948 |  반복 1 / 2 | 시간 3[s] | 손실 0.41
| 에폭 949 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 950 |  반복 1 / 2 | 시간 3[s] | 손실 0.32
| 에폭 951 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 952 |  반복 1 / 2 | 시간 3[s] | 손실 0.18
| 에폭 953 |  반복 1 / 2 | 시간 3[s] | 손실 0.33
| 에폭 954 |  반복 1 / 2 | 시간 3[s] | 손실 0.31
| 에폭 955 |  반복 1

<Figure size 640x480 with 1 Axes>

you [ 1.6806451  -0.9932009  -0.98569196 -1.0287186  -1.007644  ]
say [-0.21459511  1.23045     1.2046273  -0.00967017  1.2078671 ]
goodbye [-0.7127211 -0.9707529 -0.8989045 -1.079927  -1.0007586]
and [-1.6069981  1.032331   1.0403984  1.5256115  1.0024816]
i [-0.6960839  -0.94845694 -0.88968116 -1.0708066  -0.96662515]
hello [ 1.6503935  -1.0198022  -0.98347485 -1.0304602  -1.0313885 ]
. [ 1.3341502   0.99960744  1.0713681  -1.3790479   0.9967154 ]
